In [ ]:
import torch
import torch.nn as nn

In [ ]:
class Generator(nn.Module):
    def __init__(self,latent_noise,channel_img,feature_g,num_classes,img_size,embedded_size):
        super().__init__()
        self.img_size=img_size
        self.gen=nn.Sequential(
                self._block(latent_noise+embedded_size,feature_g*16,4,1,0),
                self._block(feature_g*16,feature_g*8,4,2,1),
                self._block(feature_g*8,feature_g*4,4,2,1),
                self._block(feature_g*4,feature_g*2,4,2,1),
                nn.ConvTranspose2d(feature_g*2,channel_img,4,2,1),
                nn.Tanh())
        self.embed=nn.Embedding(num_classes,embedded_size)
        
    def _block(self,in_channel,out_channel,kernel,stride,padding):
        return nn.Sequential(
                nn.ConvTranspose2d(in_channel,out_channel,kernel,stride,padding),
                nn.BatchNorm2d(out_channel),
                nn.ReLU())
    
    def forward(self,x,labels):
        embedding=self.embed(labels).unsqueeze(2).unsqueeze(3)
        x=torch.cat([x,embedding],dim=1)
        return self.gen(x)